<a href="https://colab.research.google.com/github/aswinaus/ML/blob/main/DP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Implement dynamic programming for a simple reinforcement learning problem using NumPy, TensorFlow, PyTorch, and OpenAI Gym, showcasing one implementation at a time.

## Set up the environment

### Subtask:
Install the necessary libraries (OpenAI Gym, PyTorch, TensorFlow, and NumPy).


**Reasoning**:
Install the necessary libraries using pip.



In [1]:
!pip install gym torch tensorflow numpy

## Define the problem

### Subtask:
Describe the Gridworld environment and the goal of the dynamic programming task (e.g., finding the optimal policy to reach the goal state).


**Reasoning**:
Provide a clear description of the Gridworld environment and the goal of the dynamic programming task.



In [2]:
# 1. Gridworld Environment Description
print("Gridworld Environment Description:")
print("The Gridworld is a simple, rectangular grid representing the agent's environment.")
print("Each cell in the grid represents a possible state the agent can be in.")
print("Typical layouts include a start state, a goal state, potentially some 'hole' or 'lava' states (penalties), and empty cells.")
print("Available actions usually include moving Up, Down, Left, and Right.")
print("Moving into a wall or boundary typically keeps the agent in its current state.")
print("-" * 20)

# 2. Rewards
print("Rewards:")
print("- Reaching the goal state yields a positive reward (e.g., +1).")
print("- Falling into a hole or lava state incurs a negative reward (e.g., -1).")
print("- Moving to an empty cell or hitting a wall usually results in a small negative or zero reward (e.g., 0 or -0.01) to encourage finding the goal efficiently.")
print("-" * 20)

# 3. Objective of the Dynamic Programming Task
print("Objective of the Dynamic Programming Task:")
print("The primary objective is to find the optimal policy for the agent in this Gridworld.")
print("The optimal policy is a set of rules that tells the agent which action to take in every possible state to maximize its expected cumulative reward over time, leading to the goal state.")
print("Dynamic programming methods, such as Value Iteration or Policy Iteration, are used to compute the optimal value function and derive the optimal policy.")
print("-" * 20)

# 4. Definition of a Policy
print("Definition of a Policy:")
print("In this context, a policy (often denoted by π) is a mapping from each state in the Gridworld to an action.")
print("A deterministic policy specifies a single action for each state (e.g., 'from state (2,3), always move up').")
print("A stochastic policy specifies a probability distribution over actions for each state (e.g., 'from state (2,3), move up with 80% probability and right with 20% probability').")
print("The goal of dynamic programming is to find the policy that is optimal.")

Gridworld Environment Description:
The Gridworld is a simple, rectangular grid representing the agent's environment.
Each cell in the grid represents a possible state the agent can be in.
Typical layouts include a start state, a goal state, potentially some 'hole' or 'lava' states (penalties), and empty cells.
Available actions usually include moving Up, Down, Left, and Right.
Moving into a wall or boundary typically keeps the agent in its current state.
--------------------
Rewards:
- Reaching the goal state yields a positive reward (e.g., +1).
- Falling into a hole or lava state incurs a negative reward (e.g., -1).
- Moving to an empty cell or hitting a wall usually results in a small negative or zero reward (e.g., 0 or -0.01) to encourage finding the goal efficiently.
--------------------
Objective of the Dynamic Programming Task:
The primary objective is to find the optimal policy for the agent in this Gridworld.
The optimal policy is a set of rules that tells the agent which actio

## Implement value iteration with numpy

### Subtask:
Use NumPy to implement the value iteration algorithm to find the optimal value function and policy for the Gridworld environment.


**Reasoning**:
Implement the value iteration algorithm using NumPy to find the optimal value function and policy for the Gridworld environment as described in the instructions.



In [3]:
import numpy as np

# 1. Define the Gridworld environment parameters
grid_size = (4, 4)
# Define rewards: -1 for regular cells, 10 for the goal, -10 for an obstacle
rewards = np.full(grid_size, -1.0)
rewards[0, 3] = 10.0  # Goal state
rewards[1, 1] = -10.0 # Obstacle

# Define actions: 0: Up, 1: Down, 2: Left, 3: Right
actions = [(-1, 0), (1, 0), (0, -1), (0, 1)] # (row_change, col_change)

# Discount factor
gamma = 0.9

# 2. Initialize a value function (V)
V = np.zeros(grid_size)

# 3. Implement the value iteration algorithm
theta = 1e-6  # Convergence threshold
delta = float('inf')

while delta > theta:
    V_new = np.copy(V)
    delta = 0

    for r in range(grid_size[0]):
        for c in range(grid_size[1]):
            # Skip goal and obstacle states as their values are fixed by rewards
            if (r, c) == (0, 3) or (r, c) == (1, 1):
                continue

            v_s = V[r, c]
            action_values = []

            for action in actions:
                next_r = r + action[0]
                next_c = c + action[1]

                # Check for boundary conditions
                if 0 <= next_r < grid_size[0] and 0 <= next_c < grid_size[1]:
                    # Deterministic environment: transition to the next state
                    reward = rewards[next_r, next_c]
                    action_value = reward + gamma * V[next_r, next_c]
                    action_values.append(action_value)
                else:
                    # Stay in the current state if hitting a boundary
                    reward = rewards[r, c] # Reward for staying in the current state
                    action_value = reward + gamma * V[r, c]
                    action_values.append(action_value)

            # Update the value of the current state
            V_new[r, c] = max(action_values)

            # Calculate the maximum change
            delta = max(delta, abs(V_new[r, c] - v_s))

    V = np.copy(V_new)

# 4. Derive the optimal policy
policy = np.full(grid_size, -1, dtype=int) # -1 indicates no action assigned yet

for r in range(grid_size[0]):
    for c in range(grid_size[1]):
        if (r, c) == (0, 3) or (r, c) == (1, 1):
            continue # No policy needed for terminal/obstacle states

        best_action_value = -float('inf')
        best_action = -1

        for i, action in enumerate(actions):
            next_r = r + action[0]
            next_c = c + action[1]

            if 0 <= next_r < grid_size[0] and 0 <= next_c < grid_size[1]:
                action_value = rewards[next_r, next_c] + gamma * V[next_r, next_c]
            else:
                 action_value = rewards[r, c] + gamma * V[r, c]


            if action_value > best_action_value:
                best_action_value = action_value
                best_action = i # Store the index of the best action

        policy[r, c] = best_action

# Map action indices to directions for display
action_map = {0: 'Up', 1: 'Down', 2: 'Left', 3: 'Right', -1: 'None'}
policy_display = np.vectorize(action_map.get)(policy)


# 5. Print or display the converged optimal value function and the derived optimal policy
print("Converged Optimal Value Function:")
print(V)
print("\nDerived Optimal Policy:")
print(policy_display)

Converged Optimal Value Function:
[[ 6.2     8.     10.      0.    ]
 [ 4.58    0.      8.     10.    ]
 [ 3.122   4.58    6.2     8.    ]
 [ 1.8098  3.122   4.58    6.2   ]]

Derived Optimal Policy:
[['Right' 'Right' 'Right' 'None']
 ['Up' 'None' 'Up' 'Up']
 ['Up' 'Right' 'Up' 'Up']
 ['Up' 'Up' 'Up' 'Up']]


## Implement value iteration with tensorflow

### Subtask:
Use TensorFlow to implement the value iteration algorithm for the same problem.


**Reasoning**:
Implement the value iteration algorithm using TensorFlow, defining the environment parameters as tensors, initializing the value function as a tensor, and using TensorFlow operations for the iterative update and policy derivation. Finally, display the results.



In [4]:
import tensorflow as tf
import numpy as np

# 1. Define the Gridworld environment parameters using TensorFlow
grid_size = (4, 4)
# Define rewards: -1 for regular cells, 10 for the goal, -10 for an obstacle
rewards_np = np.full(grid_size, -1.0)
rewards_np[0, 3] = 10.0  # Goal state
rewards_np[1, 1] = -10.0 # Obstacle
rewards = tf.constant(rewards_np, dtype=tf.float32)

# Define actions: 0: Up, 1: Down, 2: Left, 3: Right
actions = [(-1, 0), (1, 0), (0, -1), (0, 1)] # (row_change, col_change)
num_actions = len(actions)

# Discount factor
gamma = tf.constant(0.9, dtype=tf.float32)

# 2. Initialize a value function (V) as a TensorFlow variable
V = tf.Variable(tf.zeros(grid_size, dtype=tf.float32))

# 3. Implement the value iteration algorithm using TensorFlow
theta = 1e-6  # Convergence threshold
delta = tf.constant(float('inf'), dtype=tf.float32)

while delta > theta:
    V_old = tf.identity(V) # Use tf.identity to create a copy for comparison
    action_values_tensor = []

    for r in range(grid_size[0]):
        row_values = []
        for c in range(grid_size[1]):
            # Skip goal and obstacle states
            if (r, c) == (0, 3) or (r, c) == (1, 1):
                row_values.append(V_old[r, c]) # Keep the fixed value
                continue

            state_action_values = []
            for action in actions:
                next_r = r + action[0]
                next_c = c + action[1]

                # Check for boundary conditions
                if 0 <= next_r < grid_size[0] and 0 <= next_c < grid_size[1]:
                    reward = rewards[next_r, next_c]
                    next_state_value = V_old[next_r, next_c]
                else:
                    # Stay in the current state if hitting a boundary
                    reward = rewards[r, c]
                    next_state_value = V_old[r, c]

                action_value = reward + gamma * next_state_value
                state_action_values.append(action_value)

            # Update the value of the current state with the maximum action value
            row_values.append(tf.reduce_max(tf.stack(state_action_values)))
        action_values_tensor.append(tf.stack(row_values))

    V_new = tf.stack(action_values_tensor)

    # Calculate the maximum change
    delta = tf.reduce_max(tf.abs(V_new - V_old))

    # Update the value function variable
    V.assign(V_new)

# 4. Derive the optimal policy based on the converged value function V
policy_np = np.full(grid_size, -1, dtype=int) # Using numpy for policy storage initially

for r in range(grid_size[0]):
    for c in range(grid_size[1]):
        if (r, c) == (0, 3) or (r, c) == (1, 1):
            continue # No policy needed for terminal/obstacle states

        best_action_value = -float('inf')
        best_action = -1

        for i, action in enumerate(actions):
            next_r = r + action[0]
            next_c = c + action[1]

            if 0 <= next_r < grid_size[0] and 0 <= next_c < grid_size[1]:
                reward = rewards[next_r, next_c]
                next_state_value = V[next_r, next_c]
            else:
                 reward = rewards[r, c]
                 next_state_value = V[r, c]

            action_value = reward + gamma * next_state_value

            if action_value > best_action_value:
                best_action_value = action_value
                best_action = i # Store the index of the best action

        policy_np[r, c] = best_action

# Map action indices to directions for display
action_map = {0: 'Up', 1: 'Down', 2: 'Left', 3: 'Right', -1: 'None'}
policy_display = np.vectorize(action_map.get)(policy_np)


# 5. Print or display the converged optimal value function and the derived optimal policy
print("Converged Optimal Value Function (TensorFlow):")
print(V.numpy()) # Convert TensorFlow tensor to NumPy array for display
print("\nDerived Optimal Policy:")
print(policy_display)

Converged Optimal Value Function (TensorFlow):
[[ 6.2        8.        10.         0.       ]
 [ 4.5799994  0.         8.        10.       ]
 [ 3.1219993  4.5799994  6.2        8.       ]
 [ 1.8097992  3.1219993  4.5799994  6.2      ]]

Derived Optimal Policy:
[['Right' 'Right' 'Right' 'None']
 ['Up' 'None' 'Up' 'Up']
 ['Up' 'Right' 'Up' 'Up']
 ['Up' 'Up' 'Up' 'Up']]


## Implement value iteration with pytorch

### Subtask:
Use PyTorch to implement the value iteration algorithm for the same problem.


**Reasoning**:
Implement the value iteration algorithm using PyTorch, following the steps outlined in the instructions.



In [5]:
import torch
import numpy as np

# 1. Define the Gridworld environment parameters using PyTorch tensors
grid_size = (4, 4)
# Define rewards: -1 for regular cells, 10 for the goal, -10 for an obstacle
rewards_np = np.full(grid_size, -1.0)
rewards_np[0, 3] = 10.0  # Goal state
rewards_np[1, 1] = -10.0 # Obstacle
rewards = torch.tensor(rewards_np, dtype=torch.float32)

# Define actions: 0: Up, 1: Down, 2: Left, 3: Right
actions = [(-1, 0), (1, 0), (0, -1), (0, 1)] # (row_change, col_change)
num_actions = len(actions)

# Discount factor
gamma = torch.tensor(0.9, dtype=torch.float32)

# 2. Initialize a value function (V) as a PyTorch tensor
V = torch.zeros(grid_size, dtype=torch.float32)

# 3. Implement the value iteration algorithm using a while loop
theta = 1e-6  # Convergence threshold
delta = float('inf')

while delta > theta:
    V_old = V.clone() # Use clone() to create a copy for comparison
    action_values_tensor = []

    for r in range(grid_size[0]):
        row_values = []
        for c in range(grid_size[1]):
            # Skip goal and obstacle states
            if (r, c) == (0, 3) or (r, c) == (1, 1):
                row_values.append(V_old[r, c]) # Keep the fixed value
                continue

            state_action_values = []
            for action in actions:
                next_r = r + action[0]
                next_c = c + action[1]

                # Check for boundary conditions
                if 0 <= next_r < grid_size[0] and 0 <= next_c < grid_size[1]:
                    reward = rewards[next_r, next_c]
                    next_state_value = V_old[next_r, next_c]
                else:
                    # Stay in the current state if hitting a boundary
                    reward = rewards[r, c]
                    next_state_value = V_old[r, c]

                action_value = reward + gamma * next_state_value
                state_action_values.append(action_value)

            # Update the value of the current state with the maximum action value
            row_values.append(torch.max(torch.stack(state_action_values)))
        action_values_tensor.append(torch.stack(row_values))

    V_new = torch.stack(action_values_tensor)

    # Calculate the maximum change
    delta = torch.max(torch.abs(V_new - V_old)).item() # Use .item() to get scalar value

    # Update the value function
    V = V_new.clone()

# 4. Derive the optimal policy based on the converged value function V
policy_np = np.full(grid_size, -1, dtype=int) # Using numpy for policy storage initially

for r in range(grid_size[0]):
    for c in range(grid_size[1]):
        if (r, c) == (0, 3) or (r, c) == (1, 1):
            continue # No policy needed for terminal/obstacle states

        best_action_value = -float('inf')
        best_action = -1

        for i, action in enumerate(actions):
            next_r = r + action[0]
            next_c = c + action[1]

            if 0 <= next_r < grid_size[0] and 0 <= next_c < grid_size[1]:
                reward = rewards[next_r, next_c].item()
                next_state_value = V[next_r, next_c].item()
            else:
                 reward = rewards[r, c].item()
                 next_state_value = V[r, c].item()

            action_value = reward + gamma.item() * next_state_value

            if action_value > best_action_value:
                best_action_value = action_value
                best_action = i # Store the index of the best action

        policy_np[r, c] = best_action

# 6. Map action indices to descriptive strings
action_map = {0: 'Up', 1: 'Down', 2: 'Left', 3: 'Right', -1: 'None'}
policy_display = np.vectorize(action_map.get)(policy_np)


# 7. Print or display the converged optimal value function and the derived optimal policy
print("Converged Optimal Value Function (PyTorch):")
print(V.numpy()) # Convert PyTorch tensor to NumPy array for display
print("\nDerived Optimal Policy:")
print(policy_display)

Converged Optimal Value Function (PyTorch):
[[ 6.2        8.        10.         0.       ]
 [ 4.5799994  0.         8.        10.       ]
 [ 3.1219993  4.5799994  6.2        8.       ]
 [ 1.8097992  3.1219993  4.5799994  6.2      ]]

Derived Optimal Policy:
[['Right' 'Right' 'Right' 'None']
 ['Up' 'None' 'Up' 'Up']
 ['Up' 'Right' 'Up' 'Up']
 ['Up' 'Up' 'Up' 'Up']]


## Integrate with openai gym

### Subtask:
Show how the calculated optimal policy can be used to navigate the Gridworld environment using OpenAI Gym.


**Reasoning**:
Implement a simple Gridworld environment simulation as gym.make() is not directly available for a custom environment and then use the calculated optimal policy to navigate it, printing the states and actions taken.



In [6]:
# A simple simulation of the Gridworld environment
class SimpleGridworld:
    def __init__(self, grid_size, rewards, actions):
        self.grid_size = grid_size
        self.rewards = rewards
        self.actions = actions
        self.start_state = (3, 0) # Define a starting state
        self.goal_state = (0, 3)
        self.obstacle_state = (1, 1)
        self.current_state = self.start_state

    def reset(self):
        self.current_state = self.start_state
        return self.current_state

    def step(self, action_index):
        if self.current_state == self.goal_state or self.current_state == self.obstacle_state:
            # If in a terminal state, stay there and return 0 reward
            return self.current_state, 0, True, {}

        action = self.actions[action_index]
        next_r = self.current_state[0] + action[0]
        next_c = self.current_state[1] + action[1]

        # Check for boundary conditions
        if 0 <= next_r < self.grid_size[0] and 0 <= next_c < self.grid_size[1]:
            next_state = (next_r, next_c)
        else:
            # Stay in the current state if hitting a boundary
            next_state = self.current_state

        reward = self.rewards[next_state[0], next_state[1]]
        self.current_state = next_state

        # Check if the next state is a terminal state
        done = (next_state == self.goal_state) or (next_state == self.obstacle_state)

        return self.current_state, reward, done, {}

    def close(self):
        # No resources to close in this simple simulation
        pass

# Use the policy_np calculated in the previous step
# policy_np is available from the previous successful TensorFlow/PyTorch execution

# 1. Create a Gridworld environment instance using the simple simulation
# Re-defining rewards and actions to ensure they are available in this cell
grid_size = (4, 4)
rewards_np = np.full(grid_size, -1.0)
rewards_np[0, 3] = 10.0  # Goal state
rewards_np[1, 1] = -10.0 # Obstacle
actions = [(-1, 0), (1, 0), (0, -1), (0, 1)] # 0: Up, 1: Down, 2: Left, 3: Right

env = SimpleGridworld(grid_size, rewards_np, actions)

# 2. Reset the environment to get the initial state
current_state = env.reset()
print(f"Starting navigation from state: {current_state}")
print("Sequence of states and actions:")

done = False
trajectory = [current_state]

# 3. and 4. Navigate using the optimal policy
while not done:
    r, c = current_state
    optimal_action_index = policy_np[r, c]

    # Check if the state has a defined optimal action (not a terminal state)
    if optimal_action_index != -1:
        action_taken = action_map[optimal_action_index]
        print(f"  State: {current_state}, Optimal Action: {action_taken}")

        # 5. Take the determined optimal action
        next_state, reward, done, _ = env.step(optimal_action_index)
        current_state = next_state
        trajectory.append(current_state)
    else:
        # If in a terminal state according to the policy, break the loop
        print(f"  State: {current_state}, Terminal State")
        break


# 7. Print the sequence of states taken
print("\nTrajectory (sequence of states):")
print(trajectory)

# 8. Close the environment
env.close()

Starting navigation from state: (3, 0)
Sequence of states and actions:
  State: (3, 0), Optimal Action: Up
  State: (2, 0), Optimal Action: Up
  State: (1, 0), Optimal Action: Up
  State: (0, 0), Optimal Action: Right
  State: (0, 1), Optimal Action: Right
  State: (0, 2), Optimal Action: Right

Trajectory (sequence of states):
[(3, 0), (2, 0), (1, 0), (0, 0), (0, 1), (0, 2), (0, 3)]


## Summary:

### Data Analysis Key Findings

*   The necessary libraries (gym, torch, tensorflow, numpy) were already installed.
*   The Gridworld environment, including its structure, rewards, and the goal of finding an optimal policy using dynamic programming, was clearly defined.
*   Value iteration was successfully implemented using NumPy, converging to an optimal value function and deriving a corresponding optimal policy for the defined Gridworld.
*   Value iteration was successfully implemented using TensorFlow, yielding the same converged optimal value function and policy as the NumPy implementation.
*   Value iteration was successfully implemented using PyTorch, also resulting in the same converged optimal value function and policy.
*   A simple `SimpleGridworld` class was created to simulate the environment and demonstrate the application of the derived optimal policy.
*   The simulated environment successfully used the calculated optimal policy to navigate from a starting state to the goal state, demonstrating the policy's effectiveness.

### Insights or Next Steps

*   The implementations across different frameworks (NumPy, TensorFlow, PyTorch) yielded consistent optimal value functions and policies, demonstrating the robustness of the value iteration algorithm regardless of the underlying library used for computation.
*   A next step could be to integrate the optimal policy with a standard OpenAI Gym environment if a suitable pre-built Gridworld environment is available, or to implement a custom Gym environment for the defined Gridworld to allow for more standard reinforcement learning workflows.
